<a href="https://colab.research.google.com/github/kmouts/PMS/blob/master/transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2018 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

Επιμέλεια: Κ.Μούτσελος Παν/μιο Πειραιώς

# Μεταφορά Μάθησης -Transfer learning (με TensorFlow Hub)



Το [TensorFlow Hub](http://tensorflow.org/hub) είναι ένα αποθετήριο προκατασκευασμένων μοντέλων. Ανατρέξτε στο [TensorFlow Module Hub](https://tfhub.dev/) για μια λίστα με προ-εκπαιδευμένα μοντέλα με δυνατότητα αναζήτησης. Σε αυτόν τον οδηγό θα δούμε:

1. Πώς να χρησιμοποιήσουμε το TensorFlow Hub με το `tf.keras`.
1. Ταξινόμηση εικόνας χρησιμοποιώντας ένα μοντέλο από το TensorFlow Hub.
1. Πώς να γίνεται απλή μάθηση μεταφορά μάθησης.

## Αρχικές ρυθμίσεις

In [0]:
import matplotlib.pylab as plt

!pip install -q tf-nightly
import tensorflow as tf

In [0]:
!pip install -q -U tf-hub-nightly
!pip install -q tfds-nightly
import tensorflow_hub as hub

from tensorflow.keras import layers

## Ένας ταξινομητής της ImageNet

### Κατεβάστε τον ταξινομητή

Use `hub.module` to load a mobilenet, and `tf.keras.layers.Lambda` to wrap it up as a keras layer. Any [TensorFlow 2 compatible image classifier URL](https://tfhub.dev/s?q=tf2&module-type=image-classification) from tfhub.dev will work here.

Χρησιμοποιούμε το `hub.module` για να φορτώσουμε ένα μοντέλο mobilenet και το `tf.keras.layers.Lambda` για να το τυλίξουμε (wrap) ως ένα επίπεδο στο keras. Μπορούμε να βάλουμε οποιοδήποτε [URL που να περιέχει ταξινομητή εικόνας με TensorFlow 2](https://tfhub.dev/s?q=tf2&module-type=image-classification) από το tfhub.dev.

In [0]:
classifier_url ="https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/2" #@param {type:"string"}

In [0]:
IMAGE_SHAPE = (224, 224)

classifier = tf.keras.Sequential([
    hub.KerasLayer(classifier_url, input_shape=IMAGE_SHAPE+(3,))
])

### Ας το τρέξουμε σε μία μόνο εικόνα

Κατεβάζουμε μία εικόνα για να δοκιμάσουμε το μοντέλο.

In [0]:
import numpy as np
import PIL.Image as Image

surfer = tf.keras.utils.get_file('image.png','https://www.tensorflow.org/tutorials/images/transfer_learning_with_hub_files/output_w5wDjXNjuXGD_0.png')
surfer = Image.open(surfer).resize(IMAGE_SHAPE)
surfer

In [0]:
surfer = np.array(surfer)/255.0
surfer.shape

Προσθέτουμε μια διάσταση παρτίδας (batch) στην αρχή του πίνακα της εικόνας:

In [0]:
surferb = surfer[np.newaxis, ...]
surferb.shape

Στέλνουμε την εικόνα στο μοντέλο για πρόβλεψη:

In [0]:
result = classifier.predict(surferb)
result.shape

Το αποτέλεσμα είναι ένα άνυσμα 1001 στοιχείων με logits, ο οποίος βαθμολογεί την πιθανότητα κάθε κλάσης για την εικόνα.

Το ID της κορυφαίας κατηγορίας μπορεί να βρεθεί με το argmax:

In [0]:
predicted_class = np.argmax(result[0], axis=-1)
predicted_class

### Aποκωδικοποίηση της πρόβλεψης

Έχουμε το ID της κλάσης που προβλέφθηκε, οπότε λαμβάνουμε όες τις ετικέτες ImageNet και αποκωδικοποιούμε την πρόβλεψη.

In [0]:
labels_path = tf.keras.utils.get_file('ImageNetLabels.txt','https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt')
imagenet_labels = np.array(open(labels_path).read().splitlines())

In [0]:
plt.imshow(surfer)
plt.axis('off')
predicted_class_name = imagenet_labels[predicted_class]
_ = plt.title("Prediction: " + predicted_class_name.title())

## Απλή μεταφορά μάθησης

Χρησιμοποιώντας το TF Hub είναι απλό να επανεκπαιδεύσουμε το τελευταίο (top) επίπεδο του μοντέλου για να αναγνωρίζει τις κλάσεις στα δικά μας δεδομένα.

### Dataset

Σε αυτό το παράδειγμα θα χρησιμοποιήσουμε το TensorFlow flowers dataset:

In [0]:
data_root = tf.keras.utils.get_file(
  'flower_photos','https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
   untar=True)

Ο απλούστερος τρόπος φόρτωσης αυτών των δεδομένων στο μοντέλο μας είναι με χρήση του `tf.keras.preprocessing.image.ImageDataGenerator`,

Όλες τα μοντέλα εικόνων στο TensorFlow Hub παίρνουν ως είσοδο  δεκαδικούς αριθμούς στην περιοχή [0, 1]. Για το σκοπό αυτό χρησιμοποιούμε τη παράμετρο `rescale` του `ImageDataGenerator`.

Το μέγεθος της εικόνας θα το χειριστούμε αργότερα.

In [0]:
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)
image_data = image_generator.flow_from_directory(str(data_root), target_size=IMAGE_SHAPE)

Το αντικείμενο που προκύπτει είναι μια γενήτρια iterator που επιστρέφει ζεύγη: `image_batch, label_batch`.

In [0]:
for image_batch, label_batch in image_data:
  print("Image batch shape: ", image_batch.shape)
  print("Label batch shape: ", label_batch.shape)
  break

### Τρέξιμο του ταξινομητή σε μια παρτίδα εικόνων

Ας προβλέψει ο ταξινομητής τις κλάσεις σε μια παρτίδα εικόνων.

In [0]:
result_batch = classifier.predict(image_batch)
result_batch.shape

In [0]:
predicted_class_names = imagenet_labels[np.argmax(result_batch, axis=-1)]
predicted_class_names

Ας ελέγξουμε πώς αυτές οι προβλέψεις ταιριάζουμε με τις εικόνες:

In [0]:
plt.figure(figsize=(10,9))
plt.subplots_adjust(hspace=0.5)
for n in range(30):
  plt.subplot(6,5,n+1)
  plt.imshow(image_batch[n])
  plt.title(predicted_class_names[n])
  plt.axis('off')
_ = plt.suptitle("ImageNet predictions")

Τα αποτελέσματα δεν είναι καθόλου τέλεια, αλλά είναι λογικά, δεδομένου ότι δεν συμπίπτουν με τις κλάσεις που εκπαιδεύτηκε το μοντέλο (εκτός από τη "daisy").

### Κατεβάστε το μοντέλο χωρίς το τελευταίο επίπεδο (headless)

Το TensorFlow Hub διανέμει επίσης μοντέλα χωρίς το τελευταίο επίπεδο ταξινόμησης. Αυτά μπορούν να χρησιμοποιηθούν για εύκολη μεταφορά μάθησης.

Οποιοδήποτε συμβατό [URL Tensorflow 2](https://tfhub.dev/s?module-type=image-feature-vector&q=tf2) από το tfhub.dev θα λειτουργεί.

In [0]:
feature_extractor_url = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/2" #@param {type:"string"}

Δημιουργήστε το επίπεδο εξαγωγής χαρακτηριστικών.

In [0]:
feature_extractor_layer = hub.KerasLayer(feature_extractor_url,
                                         input_shape=(224,224,3))

Για κάθε εικόνα επιστρέφει ένα διάνυσμα μήκους 1280 στοιχείων:

In [0]:
feature_batch = feature_extractor_layer(image_batch)
print(feature_batch.shape)

Παγώστε τις μεταβλητές στο επίπεδο εξαγωγής χαρακτηριστικών, ώστε η εκπαίδευση να τροποποιεί μόνο το νέο επίπεδο του ταξινομητή.

In [0]:
feature_extractor_layer.trainable = False

### Πρόσθεση κεφαλής ταξινόμησης (classification head)

Τώρα τυλίγουμε το  hub  σε ένα σειριακό μοντέλο `tf.keras.Sequential` και προσθέτουμε ένα νέο επίπεδο ταξινόμησης.

In [0]:
model = tf.keras.Sequential([
  feature_extractor_layer,
  layers.Dense(image_data.num_classes)
])

model.summary()

In [0]:
predictions = model(image_batch)

In [0]:
predictions.shape

### Εκπαίδευση του μοντέλου

Χρησιμοποιούμε τη συνάρτηση compile για να διαμορφώσουμε τη διαδικασία εκπαίδευσης:

In [0]:
model.compile(
  optimizer=tf.keras.optimizers.Adam(),
  loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
  metrics=['acc'])

Και τώρα χρησιμοποιούμε τη μέθοδο `.fit` method για να εκπαιδεύσουμε το μοντέλο.

Για να διατηρήσουμε αυτό το παράδειγμα σύντομο, ας το τρέξουμε μόνο για 2 εποχές. Για να απεικονίσουμε την πρόοδο της εκπαίδευσης, ας χρησιμοποιούσουμε μια προσαρμοσμένη συνάρτηση callback για να καταγράφει την απώλεια (loss) και την ακρίβεια (accuracy) κάθε παρτίδας ξεχωριστά, αντί για τον μέσο όρο της εποχής.

In [0]:
class CollectBatchStats(tf.keras.callbacks.Callback):
  def __init__(self):
    self.batch_losses = []
    self.batch_acc = []

  def on_train_batch_end(self, batch, logs=None):
    self.batch_losses.append(logs['loss'])
    self.batch_acc.append(logs['acc'])
    self.model.reset_metrics()

In [0]:
steps_per_epoch = np.ceil(image_data.samples/image_data.batch_size)

batch_stats_callback = CollectBatchStats()

history = model.fit_generator(image_data, epochs=2,
                              steps_per_epoch=steps_per_epoch,
                              callbacks = [batch_stats_callback])

Now after, even just a few training iterations, we can already see that the model is making progress on the task.
Aκόμη και με τόσο λίγες επαναλήψεις εκπαίδευσης, μπορούμε  να δούμε ήδη ότι το μοντέλο σημειώνει πρόοδο.

In [0]:
plt.figure()
plt.ylabel("Loss")
plt.xlabel("Training Steps")
plt.ylim([0,2])
plt.plot(batch_stats_callback.batch_losses)

In [0]:
plt.figure()
plt.ylabel("Accuracy")
plt.xlabel("Training Steps")
plt.ylim([0,1])
plt.plot(batch_stats_callback.batch_acc)

### Έλεγχος προβλέψεων

Για να ξανατρέξουμε το προηγούμενο γράφημε, πρώτα παίρνουμε μια ταξινομημένη λίστα των ονομάτων των κλάσεων:

In [0]:
class_names = sorted(image_data.class_indices.items(), key=lambda pair:pair[1])
class_names = np.array([key.title() for key, value in class_names])
class_names

Τρέχουμε τη παρτίδα εικόνων από το μοντέλο και μετατρέπουμε τους αριθμούς σε ονόματα κλάσεων.

In [0]:
predicted_batch = model.predict(image_batch)
predicted_id = np.argmax(predicted_batch, axis=-1)
predicted_label_batch = class_names[predicted_id]

Ας δούμε τα αποτελέσματα:

In [0]:
label_id = np.argmax(label_batch, axis=-1)

In [0]:
plt.figure(figsize=(10,9))
plt.subplots_adjust(hspace=0.5)
for n in range(30):
  plt.subplot(6,5,n+1)
  plt.imshow(image_batch[n])
  color = "green" if predicted_id[n] == label_id[n] else "red"
  plt.title(predicted_label_batch[n].title(), color=color)
  plt.axis('off')
_ = plt.suptitle("Model predictions (green: correct, red: incorrect)")

## Εξαγωγή μοντέλου

Τώρα που έχετε εκπαιδεύσει το μοντέλο, μπορείτε να το εξαχθεί και να αποθηκευτεί:

In [0]:
import time
t = time.time()

export_path = "/tmp/saved_models/{}".format(int(t))
model.save(export_path, save_format='tf')

export_path

Ας επιβεβαιώσουμε ότι μπορούμε να το ξαναφορτώσουμε και ότι εξακολουθεί να δίνει τα ίδια αποτελέσματα:

In [0]:
reloaded = tf.keras.models.load_model(export_path)

In [0]:
result_batch = model.predict(image_batch)
reloaded_result_batch = reloaded.predict(image_batch)

In [0]:
abs(reloaded_result_batch - result_batch).max()

Αυτό το αποθηκευμένο μοντέλο μπορεί να φορτωθεί για προγνώσεις αργότερα ή να μετατραπεί σε [TFLite](https://www.tensorflow.org/lite/convert/) ή [TFjs](https://github.com/tensorflow/tfjs-converter).


## Συνέχεια εξερεύνησης...



### Γραφική επόπτευση

Όταν κατασκευάζουμε μοντέλα, είναι πολύ χρήσιμο να έχουμε μια οπτική της όλης αρχιτεκτονικής. 

Η συνάρτηση `plot_model` από το `tensorflow.keras.utils` μπορεί να χρησιμοποιηθεί για να έχουμε μια οπτική σύνοψη του μοντέλου:

In [0]:
from tensorflow.keras.utils import plot_model

plot_model(model, to_file='conv_base.png', show_shapes=True)
from IPython.display import Image
Image(filename='conv_base.png')

Για να επιβεβαιώσουμε ποιά επίπεδα μπορούν να εκπαιδευτούν, μπορούμε να ελέγξουμε την ιδιότητα `trainable` του κάθε επιπέδου:

In [0]:
    # Check the trainable status of the individual layers
for layer in model.layers:
    print(layer, layer.trainable)

Με μια ανάλογη επανάληψη, μπορούμε να επιτρέπουμε ή όχι την εκπαίδευση κάποιου επιπέδου, θέτοντας πχ `layer.trainable = False` ή `True` αντίστοιχα.

Δοκιμάστε να ξαναεκπαιδεύσετε το μοντέλο, ξαπαγώνοντας όλα του τα επίπεδα. Είναι καλύτερα τα αποτελέσματα πρόβλεψης;